In [3]:
!pip install requests beautifulsoup4



In [2]:
!pip install pymysql



     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.9 MB/s eta 0:00:00


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from sqlalchemy import create_engine, text
import time

# Setup database connection
DATABASE_TYPE = 'mysql'
DBAPI = 'pymysql'
HOST = 'spacex-project1.c9mbbltare10.us-east-1.rds.amazonaws.com'
USER = 'spaceboy'
PASSWORD = 'Flytweet04'
DATABASE = 'sql_project'
PORT = 3306
engine = create_engine(f"{DATABASE_TYPE}+{DBAPI}://{USER}:{PASSWORD}@{HOST}:{PORT}/{DATABASE}")

def scrape_spacex_launches():
    base_url = 'https://www.spacex.com/launches/'
    response = requests.get(base_url)
    soup = BeautifulSoup(response.text, 'html.parser')

    # Scrape and store main launch data
    missions = soup.select('.label')
    dates = soup.select('.date')
    launch_data = [(mission.text.strip(), date.text.strip()) for mission, date in zip(missions, dates)]
    df_launches = pd.DataFrame(launch_data, columns=['mission_name', 'launch_date'])

    # Scrape details from each launch page
    launch_details = []
    links = soup.find_all('a', class_='article-header', href=True)
    for link in links:
        url = 'https://www.spacex.com' + link['href']
        detail_response = requests.get(url)
        detail_soup = BeautifulSoup(detail_response.text, 'html.parser')
        paragraphs = detail_soup.find_all('p')
        details = ' '.join(paragraph.text.strip() for paragraph in paragraphs)
        launch_details.append({'url': url, 'details': details})
    df_details = pd.DataFrame(launch_details)

    # Insert data into the database
    with engine.connect() as connection:
        df_launches.to_sql('spacex_launches', con=connection, if_exists='append', index=False)
        df_details.to_sql('spacex_launch_details', con=connection, if_exists='append', index=False)

if __name__ == "__main__":
    scrape_spacex_launches()


In [24]:
with engine.begin() as connection:  # begin a transaction
    df.to_sql('spacex_launches', con=connection, index=False, if_exists='append', method=None)


In [ ]:
!pip install selenium
!pip install webdriver_manager
